In [4]:
# import internal .py files
import file_path_management as fpath
import public_library as plib

In [5]:
# all the file paths in file_path_management.py
# project_folder
# gs_poten_urls
# wos_poten_urls
# pubmed_pmc_poten_urls
# eupmc_poten_urls
# path_poten_csv
# path_related_urls
# path_related_csv
# pdf_folder_path
# seed_paper_urls
# connec_db_urls

In [6]:
# import packages
import requests
from bs4 import BeautifulSoup
import pandas as pd
import random
import re
import time

In the next cell, we present all parameters that might have an effect on the search results, including:<br>
1. searching keyword lexicon
2. academic databases
3. initial urls when searching academic databases
4. seed paper list for spanning citations
5. conenctome database
6. seaching queries of the connectome database
7. on-topic keyword lexicon
8. weights of on-topic keywords when calculating relatedness of a literature
9. ChatGPT queries for relatedness of topic
10. meta categories when extracting information of related literature
11. keywords for searching meta categories
12. ChatGPT queries for extracting information of meta categories of related literature

In [7]:
# parameters

# searching keyword lexicon
search_kws_lexicon = "macaque AND (thalamus OR thalamocortical OR thalamo-cortical)" # in all fields

# academic databases
# Google Scholar: "https://scholar.google.com/"
# 78300 results
# Web of Science: "https://www.webofscience.com/wos/woscc/advanced-search" # can be exported to excel file
# 961 results
# PubMed Central PMC: "https://pubmed.ncbi.nlm.nih.gov/advanced/" # can be exported to .csv file and abstract.txt file
# 2448 results
# Europe PMC = "https://europepmc.org/advancesearch" # can be exported to .csv file or abstract and full open access file .xml
# 5129 results
acad_dbs = ["Google Scholar", "Web of Science", "PubMed_Central_PMC", "Europe_PMC"]

# initial urls for specified searching keyword lexicon and all academic databases
init_urls = {
    "gs": "https://scholar.google.com/scholar?start=0&q=macaque+thalamus+OR+thalamocortical+OR+thalamo-cortical&hl=en&as_sdt=1,5",
    "wos": "https://www.webofscience.com/wos/woscc/summary/79530a3c-47d5-4dd0-9b7d-b1d92fd11882-98d8472a/relevance/1",
    "pubmed": "https://pubmed.ncbi.nlm.nih.gov/?term=(((thalamus)%20OR%20(thalamocortical))%20OR%20(thalamo-cortical))%20AND%20(macaque)&sort=&page=1",
    "eupmc": "https://europepmc.org/search?query=%28%22macaque%22%20AND%20%28%22thalamus%22%20OR%20%22thalamocortical%22%20OR%20%22thalamo-cortical%22%29%20%29%20AND%20%28LANG%3A%22eng%22%20OR%20LANG%3A%22en%22%20OR%20LANG%3A%22us%22%29&page=1"
}

# seed papers specification
seed_papers = []

# connectome database and queries specification
# we search the CoCoMac
connec_db = ""
connec_db_quries = []

In [30]:
def search_google_scholar(init_url):
    # create a .txt file to record the urls of google scholar search results, clear the file if already exists
    f = open(fpath.gs_poten_urls, "w")
    f.truncate()
    f.close()

    # request the first page and extract the number of pages of the search results
    first_page = init_url
    response = requests.get(first_page, headers = plib.headers)
    soup = BeautifulSoup(response.content,"lxml")
    # print(soup)
    num_results_str_list = soup.select("div", {"class": "gs_ab_mdw"})
    for item in num_results_str_list:
        if "results" in item.get_text():
            num_results_str = item.get_text().split()
    num_results_str = num_results_str[1]
    # print(num_results_str)
    # print(int(num_results_str))
    num_results = int(re.sub(r"[^\w\s]", "", num_results_str))
    pages = int(num_results/10)
    print("Google Scholar searched " + str(num_results) + " results" + " dispalyed in " + str(pages) + " pages.")
    
    # iterate all pages and record the results
    # pages = 5
    for page in range(pages):
        time.sleep(random.randint(1, 10))
        start = page * 10
        # google scholar
        page_url = init_url.split("?start=")[0] + "?start=" + str(start) + "&q=" + init_url.split("?start=")[1].split("&q=")[1]
        # print(page_url)
        # search a page
        response = requests.get(page_url, headers = headers)
        soup = BeautifulSoup(response.content,"lxml")
        # print(soup)
        # print(soup.select("[data-lid]")) 
        for item in soup.select("[data-lid]"):
            # print(item)
            add_url = item.select("h3")[0].select("a", href = True)[0]["href"]
            # print(add_url)
            try: 
                with open(fpath.gs_poten_urls, "a") as url_file:
                    # append text at the end of file
                    url_file.write(f"{add_url}\n")
            except Exception as e: 
                print("Error when trying to write in google_scholar_poten_urls.txt")
                raise e
    print("Searching Google Scholar complated!")

def search_webofscience(init_url, headers):
    print("Searching Web of Science complated!")

def search_PubMed_Central_PMC(iinit_url, headers):
    print("Searching PubMedd Central PMC complated!")

def search_Europe_PMC(init_url, headers):
    print("Searching Europe PMC complated!")

# --------------------start of test code--------------------
init_url = "https://scholar.google.com/scholar?start=0&q=macaque+thalamus+OR+thalamocortical+OR+thalamo-cortical&hl=en&as_sdt=1,5"
headers = plib.headers
search_google_scholar(init_url)
# ---------------------end of test code---------------------

Google Scholar searched 78300 results dispalyed in 7830 pages.
Searching Google Scholar complated!


In [9]:
def search_PubMed_Central_PMC(init_url, headers):
    print("Searching PubMedd Central PMC complated!")

# --------------------start of test code--------------------
# test code
# ---------------------end of test code---------------------

In [10]:
def search_Europe_PMC(init_url, headers):
    print("Searching Europe PMC complated!")

# --------------------start of test code--------------------
# test code
# ---------------------end of test code---------------------

In [11]:
# search academic databases, record the urls as a line in a .txt file from the webpages
def search_acad_dbs(acad_dbs, init_urls, headers, proxy):
    for acad_db in acad_dbs:
        if acad_db == "Google Scholar":
            print("Searching Google Scholar...")
            search_google_scholar(init_urls["gs"], headers)
        elif acad_db == "Web of Science":
            print("Searching Web of Science...")
            search_webofscience(init_urls["wos"], headers)
        elif acad_db == "PubMed_Central_PMC":
            print("Searching PubMed Central PMC...")
            search_PubMed_Central_PMC(init_urls["pubmed"], headers)
        elif acad_db == "Europe_PMC":
            print("Searching Europe PMC...")
            search_Europe_PMC(init_urls["pubmed"], headers)
        else:
            print("The specified academic database: " + acad_db + " is not supported by this function.")
            print("Plese choose one of the following databases:",)
            for db in ["Google Scholar", "Web of Science", "PubMed_Central_PMC", "Europe_PMC"]:
                print(db)

def span_citations(seed_papers, num_span_time, headers, proxy):
    None
    
def search_conne_db(connec_db, connec_db_quries):
    None

# --------------------start of test code--------------------
# test code
# ---------------------end of test code---------------------

In [12]:
def merge_search_results():
    # process gs_poten_urls
    with open(fpath.gs_poten_urls, "r") as file:
        lines = []
        for line in file:
            print(line)
            line = line.strip()
            lines.append(line)
    print(len(lines))
    doi_list = []
    for url in lines:
        response = requests.get(url, headers = plib.headers)
        soup = BeautifulSoup(response.content,"lxml")
        # print(soup)
        num_results_str = soup.select("a", href = True)
        for href in num_results_str:
            if "//doi.org/" in href:
                doi_list.append(href)
    doi_df = pd.DataFrame({"DOI": doi_list})
    plib.clear_file(fpath.path_poten_csv)
    doi_df.to_csv(fpath.path_poten_csv, index=False)
    
    # process wos_poten_urls
    doi_df = pd.read_csv(fpath.wos_poten_urls, sep=";")
    # print(doi_df.columns)
    # print(doi_df.head())
    doi_df = doi_df[["DOI"]]
    doi_df.to_csv(fpath.path_poten_csv, mode="a", index=False, header=False)
    
    # process pubmed_pmc_poten_urls
    doi_df = pd.read_csv(fpath.pubmed_pmc_poten_urls)
    doi_df = doi_df[["DOI"]]
    doi_df.to_csv(fpath.path_poten_csv, mode="a", index=False, header=False)
    
    # process eupmc_poten_urls
    doi_df = pd.read_csv(fpath.eupmc_poten_urls)
    doi_df = doi_df[["DOI"]]
    doi_df.to_csv(fpath.path_poten_csv, mode="a", index=False, header=False)
    
    # eliminate duplicates
    doi_df = pd.read_csv(fpath.path_poten_csv)
    print(len(doi_df))
    doi_df = doi_df.drop_duplicates(subset = "DOI")
    print(len(doi_df))
    doi_df.to_csv(fpath.path_poten_csv, index=False)
    # end of merge_search_results

# --------------------start of test code--------------------
# test code
# ---------------------end of test code---------------------

In [13]:
# main program
# first we need to search all related literature that might include data or information of thalamocortical connections
# search for potentially related literature using the listed 3 methods

# # method 1: search acdemic databases using keywords
# search_acad_dbs(acad_dbs, init_urls, plib.headers, plib.proxy)

# # method 2: spanning citations of seed papers
# span_citations(seed_papers, num_span_time, headers, proxy)

# # method 3: search existing connectome databases
# search_conne_db(connec_db, connec_db_quries)

In [14]:
# merge all search results
merge_search_results(headers)

# send .PDF publication of all potential related literatures to ChatPDF.con and ask for relatedness 
# then record the answer to the list_of_potential_related_literature.csv as well
# ChatPDF_relatedness(path_urls, chatpdf_related_queries)

# now we have a list of potential related literature and the information about relatedness 
# stored in the file "list_of_potential_related_literature.csv"
# now we may perform a automatic filtering and manual filtering of the literature

# automatic filtering
#auto_filter(path_potential)

# manual filtering
# manual_filter(path_potential, path_related_urls)

0
8538
7058


This is the end of semi-automated literature search.

Now we have a list of actually related literature stored in list_of_related_literature.txt

Next step: we perform a information search on the list of related literature
We have a list of actually related literature at the moment, now we need to extract information we need from the literature. We intend to achieve this with a combination of automated searching and manual extraction

In [15]:
# some test code, should comment-out

In [16]:
# # test the redirect of the urls
# headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9"}

# response_pdf = requests.get("https://doi.org/10.1016/j.neuron.2020.01.005", allow_redirects=True, headers = headers)
# print(response_pdf.history)
# print(response_pdf.url)
# response_pdf_1 = requests.get("https://linkinghub.elsevier.com/retrieve/pii/S0896627320300052", allow_redirects=True, headers = headers)
# print(response_pdf_1.history)
# print(response_pdf_1.url)


# response_pdf = requests.get("https://onlinelibrary.wiley.com/doi/10.1111/ejn.13910", headers = headers)
# soup_pdf = BeautifulSoup(response_pdf.content,"lxml")
# print(soup_pdf)

In [17]:
# import re
# if "//doi.org/" in "https://doi.org/10.1016/0165-0173(96)00003-3":
#     print("yes")

In [21]:
# test extract doi from url
with open(fpath.gs_poten_urls, "r") as file:
    lines = []
    for line in file:
        print(line)
        line = line.strip()
        lines.append(line)
print(len(lines))
doi_list = []
for url in lines:
    response = requests.get(url, headers = headers)
    soup = BeautifulSoup(response.content,"lxml")
    # print(soup)
    num_results_str = soup.select("a", href = True)
    print(num_results_str)
    for href in num_results_str:
        if "//doi.org/" in href["href"]:
            doi_list.append(href["href"])
            print(href["href"])
        else:
            print("Ops! Did't find DOI on this page!")

0


In [ ]:
# find DOI
# this link does not have "DOI" in href form but text from
url = "https://www.jneurosci.org/content/28/43/11042.short"
# url = "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2613515/"
response = requests.get(url, headers = plib.headers)
soup = BeautifulSoup(response.content,"lxml")
# print(soup)
doi_list = []
num_results_str = soup.select("a", href = True)
# print(num_results_str)
for item in num_results_str:
    if "//doi.org/" in item["href"]:
        print(item["href"])
        doi_list.append(item["href"].split("//doi.org/")[1])

print(doi_list)
        
if len(doi_list) == 0:
    print("Ops! Did't find DOI on this page!")

[]
Ops! Did't find DOI on this page!


In [ ]:
# test redirect when access the doi link
from elsapy.elsdoc import FullDoc, AbsDoc
from elsapy.elsclient import ElsClient
import json
headers = {
    "User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9", 
    "X-ELS-APIKEY": "310946e6e005957982c2c9cad6833ad3",
    "Accept": "application/pdf",
    "X-ELS-Insttoken": "instToken",
    "view": "FULL"
} 
# url = "https://www.jneurosci.org/content/28/43/11042.short"
 #url = "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2613515/"

# Journal of Neurophysiology
# url = "https://doi.org/10.1152/jn.2001.85.1.219"
# url = "https://journals.physiology.org/doi/10.1152/jn.2001.85.1.219"

# science direct
# url = "https://doi.org/10.1016/j.biopsych.2004.10.014"
# url = "https://linkinghub.elsevier.com/retrieve/pii/S0006322304010947"
# url = "https://www.sciencedirect.com/science/article/pii/S0006322304010947?via%3Dihub"
url = "https://api.elsevier.com/content/article/doi/{10.1016/j.biopsych.2004.10.014}"

# response = requests.get(url, headers = headers)
# soup = BeautifulSoup(response.content,"lxml")
# print(soup)
# print(response.history)
# print(response.url)
# # Load configuration
# con_file = open("config.json")
# config = json.load(con_file)
# con_file.close()

# response = requests.get(url, headers = headers)
# print(response)

# ## Initialize client
# client = ElsClient(config["apikey"])

# ## ScienceDirect (full-text) document example using DOI
# doi_doc = FullDoc(doi = "10.1016/j.biopsych.2004.10.014")
# print(doi_doc)
# if doi_doc.read(client):
#     print ("doi_doc.title: ", doi_doc.title)
#     doi_doc.write("doi_doc")   
# else:
#     print ("Read document failed.")

In [ ]:
# test pdf to text
from PyPDF2 import PdfReader
  
# creating a pdf reader object
reader = PdfReader("/Users/didihou/Downloads/fncir-09-00079.pdf")
  
# printing number of pages in pdf file
print(len(reader.pages))
  
# getting a specific page from the pdf file
page = reader.pages[0]
  
# extracting text from page
text = "".join(page.extract_text().splitlines())
print(text)

8
MINI REVIEWpublished: 02 December 2015doi: 10.3389/fncir.2015.00079Vestibular Interactions in theThalamusRajiv Wijesinghe1,Dario A. Protti2andAaron J. Camp1*1Sensory Systems and Integration Laboratory, Sydney Medical School, Discipline of Biomedical Science, University ofSydney, Sydney, NSW, Australia,2Vision Laboratory, Sydney Medical School, Discipline of Physiology, University of Sydney,Sydney, NSW, AustraliaEdited by:W. Martin Usrey,University of California, Davis, USAReviewed by:Elizabeth Quinlan,University of Maryland College Park,USAMarianne Dieterich,Ludwig-Maximilians-University,Germany*Correspondence:Aaron J. Campaaron.camp@sydney.edu.auReceived: 28 August 2015Accepted: 10 November 2015Published: 02 December 2015Citation:Wijesinghe R, Protti DA and Camp AJ(2015) Vestibular Interactions inthe Thalamus.Front. Neural Circuits 9:79.doi: 10.3389/fncir.2015.00079It has long been known that the vast majority of all information en route to the cerebralcortex must ﬁrst pass through 